# **Libraries** 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import FloatSlider, HBox, VBox, Text, interactive_output, interact
from PIL import Image

<div class="alert alert-block alert-info">
Graph may be displayed multiple times. Not sure how to fix this. Please ignore. Thanks.
</div>

# **Basic 2D Transformation** 

## ***Visualisation Method***

In [ ]:
def visualise_2d(sx=1, sy=1, ax=0, ay=0, tx=0, ty=0, r=0):    
    # Base grid to modify
    x = np.linspace(-5, 5, 10)  # 10 points across -2 and 2 for x-axis
    y = np.linspace(-5, 5, 10)  # 10 points across -2 and 2 for y-axis
    X, Y = np.meshgrid(x, y)    # creates matrices for 10x10 grid
    points = np.vstack([X.flatten(), Y.flatten()])

    # (1) Create Homogeneous Matrix 
    T_mtx = np.array([
        [sx, ax, tx],
        [ay, sy, ty],
        [0,0,1]
    ])
    cos_theta = np.cos(np.radians(r))   # cos(0)=1, sin(0)=0
    sin_theta = np.sin(np.radians(r))   # when theta=0, R_mtx is identity matrix
    R_mtx = np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta,cos_theta,0],
        [0,0,1]
    ])

    # (2) Matrix Multiplication
    hg_points = np.vstack([points, np.ones(points.shape[1])])
    hg_transformed = T_mtx @ R_mtx @ hg_points
    X_src = hg_transformed[0].reshape(X.shape) 
    Y_src = hg_transformed[1].reshape(Y.shape)

    # Display matrix and transformed grid
    # Create 2 subplots to display matrix and graph
    fig, axs = plt.subplots(1, 2, figsize=(12, 7))
    
    # (1) Display Matrix Applied
    axs[0].axis("off")
    # axs[0].set_title("Transformation Applied", fontsize=12, fontweight='bold')
    
    # Display both the matrix and the transformation equation
    transformation_x = ""
    transformation_y = ""
    gap = ""

    if sx!=1 or sy!=1:
        transformation_x = f"sₓ = {sx:>5.2f}" if sx!=1 else "          "
        transformation_y = f"sᵥ = {sy:>5.2f}" if sy!=1 else "          "
        gap = "   "
    if ax!=0 or ay!=0:
        transformation_x = transformation_x + gap + f"aₓ = {ax:>5.2f}" if ax!=0 else "          "
        transformation_y = transformation_y + gap + f"aᵥ = {ay:>5.2f}" if ay!=0 else "          "
        gap = "   "

    translation_header, translation_x, translation_y, translation_footer = "","","","" 
    if tx!=0 or ty!=0:
        transformation_x = transformation_x + gap + f"tₓ = {tx:>5.2f}" if tx!=0 else "          "
        transformation_y = transformation_y + gap + f"tᵥ = {ty:>5.2f}" if ty!=0 else "          "
        translation_header =  "   ┌     ┐"
        translation_x      = f"   │{tx:>5.2f}│"
        translation_y      = f" + │{ty:>5.2f}│"
        translation_footer =  "   └     ┘"
    
    rotationA_header, rotationA_x, rotationA_y, rotationA_footer = "","","","" 
    rotationH_header, rotationH_x, rotationH_y, rotationH_z, rotationH_footer = "","","","","" 
    if r!=0:
        transformation_x += ("   " if len(transformation_x)!=0 else "") + f"θ = {r:>3.0f}⁰"  
        transformation_y += ("   " if len(transformation_x)!=0 else "") + f"        "   
        
        angle = f"{r:>3.0f}⁰"
        rotationA_header =  " ┌                    ┐"
        rotationA_x      = f" │cos({angle}) -sin({angle})│"
        rotationA_y      = f" │sin({angle})  cos({angle})│"
        rotationA_footer =  " └                    ┘"

        rotationH_header =  " ┌                       ┐"
        rotationH_x      = f" │cos({angle}) -sin({angle})  0│"
        rotationH_y      = f" │sin({angle})  cos({angle})  0│"
        rotationH_z     =   " │      0          0    1│"
        rotationH_footer =  " └                       ┘"

    text_content = [
        "Transformation:",
        transformation_x,
        transformation_y,
        "",
        "Matrix:",
        f"┌             ┐"          +rotationA_header+" ┌ ┐" +translation_header,
        f"│{sx:>5.2f}   {ax:>5.2f}│"+rotationA_x     +" │x│" +translation_x,
        f"│{ay:>5.2f}   {sy:>5.2f}│"+rotationA_y     +" │y│" +translation_y,
        f"└             ┘"          +rotationA_footer+" └ ┘" +translation_footer,
        "",
        "Homogenous",
        f"┌                     ┐"                +rotationH_header+" ┌ ┐",
        f"│{sx:>5.2f}   {ax:>5.2f}   {tx:>5.2f}│" +rotationH_x+" │x│",
        f"│{ay:>5.2f}   {sy:>5.2f}   {ty:>5.2f}│" +rotationH_y+" │y│",
        f"│    0       0       1│"                +rotationH_z+" │1│",
        f"└                     ┘"                +rotationH_footer+" └ ┘"
    ]
    
    axs[0].text(0.5, 0.6, "\n".join(text_content), 
                  fontsize=12, 
                  ha='center', va='center',
                  transform=axs[0].transAxes,
                  fontfamily='monospace',
                  linespacing=1.5)
    
    # (2) Display original and transformed graphs
    # Original Grid
    axs[1].plot(X, Y, color='gray', lw=0.5)
    axs[1].plot(X.T, Y.T, color='gray', lw=0.5)
    
    # Transformed Grid
    axs[1].plot(X_src, Y_src, color='royalblue', lw=0.7)        # horizontal
    axs[1].plot(X_src.T, Y_src.T, color='orangered', lw=0.7)    # vertical
    axs[1].plot(X_src[-1, :], Y_src[-1, :], color='limegreen', lw=1.5)   # top border
    axs[1].plot(X_src.T[-1, :], Y_src.T[-1, :], color='magenta', lw=1.5) # right border

    # Graph Size
    axs[1].set_title("Transformed Grid", fontsize=12, fontweight='bold')
    axs[1].set_xticks([])
    axs[1].set_xlim(-10, 10)
    axs[1].set_yticks([])
    axs[1].set_ylim(-10, 10)
    axs[1].set_aspect('equal')

## 1. **Scale**

In [ ]:
@interact(
    sx=FloatSlider(value=1, min=-2, max=2, step=0.25, description='Scale X'),
    sy=FloatSlider(value=1, min=-2, max=2, step=0.25, description='Scale Y')
)
def scale_2d(sx, sy):
    visualise_2d(sx=sx, sy=sy)

## 2. **Reflect**

In [ ]:
@interact(
    sx=FloatSlider(value=1, min=-1, max=1, step=2, description='Reflect X'),
    sy=FloatSlider(value=1, min=-1, max=1, step=2, description='Reflect Y')
)
def reflect_2d(sx, sy):
    visualise_2d(sx=sx, sy=sy)

## 3. **Shear**

In [ ]:
@interact(
    ax=FloatSlider(value=0, min=-4, max=4, step=0.25, description='Shear X'),
    ay=FloatSlider(value=0, min=-4, max=4, step=0.25, description='Shear Y')
)
def shear_2d(ax, ay):
    visualise_2d(ax=ax, ay=ay)

## 4. **Rotate**

In [ ]:
@interact(
    r=FloatSlider(value=0, min=0, max=360, step=10, description='Rotate')
)
def rotate_2d(r):
    visualise_2d(r=r)

## 5. **Translate**

In [ ]:
@interact(
    tx=FloatSlider(value=0, min=-5, max=5, step=0.25, description='Translate X'),
    ty=FloatSlider(value=0, min=-5, max=5, step=0.25, description='Translate Y')
)
def rotate_2d(tx, ty):
    visualise_2d(tx=tx, ty=ty)

## 6. **All**
<b>Note</b>: Rotation is done first 

In [ ]:
# Sliders
sx=FloatSlider(value=1, min=-2, max=2, step=0.25, description='Scale X')
sy=FloatSlider(value=1, min=-2, max=2, step=0.25, description='Scale Y')
ax=FloatSlider(value=0, min=-4, max=4, step=0.25, description='Shear X')
ay=FloatSlider(value=0, min=-4, max=4, step=0.25, description='Shear Y')
r=FloatSlider(value=0, min=0, max=360, step=10, description='Rotate')
tx=FloatSlider(value=0, min=-5, max=5, step=0.25, description='Translate X')
ty=FloatSlider(value=0, min=-5, max=5, step=0.25, description='Translate Y')

# Arrange layout
row1 = HBox([sx, sy])
row2 = HBox([ax, ay])
row3 = HBox([tx, ty])
row4 = HBox([r])
ui = VBox([row1, row2, row3, row4])

out = interactive_output(
    visualise_2d,
    {'sx': sx, 'sy': sy, 'ax': ax, 'ay': ay,
     'tx': tx, 'ty': ty, 'r': r}
)

VBox([ui, out])

# **Image Manipulation**

In [ ]:
def manipulate_image(img_path, sx=1, sy=1, ax=0, ay=0, tx=0, ty=0, r=0):
    try:                                # Try load picture if given
        img = Image.open(img_path)
        img_loaded=True
    except Exception as e:              # Create mock image if failed or not given
        img = np.zeros((1000, 1000, 4))         # RGBA: A = opacity
        img[450:550, 350:650] = [255, 0, 0]     # red square
        img[350:650, 450:550] = [0, 255, 0]     # green square
        img[450:550, 450:550] = [255, 255, 255] # white square
        img_loaded = False
    
    img_array = np.array(img)
    img_height, img_width, img_colour = img_array.shape

    padding = 25
    display_width = int(img_width * 2 + padding)
    display_height = int(img_height * 2 + padding)

    X_dest, Y_dest = np.meshgrid(np.arange(display_width), np.arange(display_height))

    # To shift image into top right quadrant
    早上好我是JohnCena = np.array([                     # move up by its height, right by its width
        [1,0,img_width/2], [0,1,img_height/2], [0,0,1]
    ])

    # To apply rotation about center of image
    我很喜欢冰淇淋 = np.array([                         # move from corner to center
        [1,0,-img_width/2], [0,1,-img_height/2], [0,0,1]
    ])
    现在我有冰淇淋 = np.array([                         # move from center to corner
        [1,0,img_width/2], [0,1,img_height/2], [0,0,1]
    ])

    # (1) Create Homogeneous Matrix 
    T_mtx = np.array([
        [sx, ax, tx], [ay, sy, ty], [0,0,1]
    ])
    cos_theta = np.cos(np.radians(r))   # cos(0)=1, sin(0)=0
    sin_theta = np.sin(np.radians(r))   # when theta=0, R_mtx is identity matrix
    R_mtx = np.array([
        [cos_theta, -sin_theta, 0], [sin_theta,cos_theta,0], [0,0,1]
    ])

    # (2) Matrix Multiplication
    # Create white/blank background
    transformed_img = np.ones((display_height, display_width, img_colour), dtype=img_array.dtype) * 255
    img_points = np.vstack([X_dest.flatten(), Y_dest.flatten(), np.ones(X_dest.size)])

    M = 早上好我是JohnCena @ 现在我有冰淇淋 @ T_mtx @ R_mtx @ 我很喜欢冰淇淋
    
    if np.linalg.det(M) == 0:
        
        if sx == 0 and sy == 0:
            transformed_img[display_height // 2, display_width // 2] = 0
        elif sx == 0:
            transformed_img[(display_height-img_height)//2:(display_height-img_height)//2+img_height, display_width // 2 + 5] = 0
        elif sy == 0:
            transformed_img[display_height // 2 + 5, (display_width-img_width)//2:(display_width-img_width)//2+img_width] = 0
    else:
        M_inv = np.linalg.inv(M)
        transformed_points = M_inv @ img_points
        X_src = np.round(transformed_points[0].reshape(X_dest.shape) ).astype(int) 
        Y_src = np.round(transformed_points[1].reshape(Y_dest.shape)).astype(int)

        # Create mask for valid coordinates (within original image bounds)
        valid_mask = (X_src >= 0) & (X_src < img_width) & (Y_src >= 0) & (Y_src < img_height)

        # Copy only valid pixels
        transformed_img[valid_mask] = img_array[
            np.clip(Y_src[valid_mask], 0, img_height-1),  
            np.clip(X_src[valid_mask], 0, img_width-1) 
        ]

    # Format original shape
    original_img = np.ones((display_height, display_width, img_colour), dtype=img_array.dtype) * 255
    original_img[img_height//2:img_height//2+img_height, img_width//2:img_width//2+img_width] = img_array

    # Display original and transformed images
    # Create 4 subplots
    fig, axs = plt.subplots(2, 2, figsize=(15, 10),
                       gridspec_kw={
                           'width_ratios': [2, 3],
                           'height_ratios': [3, 3],
                       'wspace': 0.4,
                       'hspace': 0.4})

    # Display original image
    axs[1,1].imshow(original_img)
    axs[1,1].set_title("Original Image")
    axs[1,1].axis('off')

    # Display transformed image
    axs[0,1].imshow(transformed_img) 
    axs[0,1].set_title("Transformed Image")
    axs[0,1].axis('off')  
    
    # Display matrix applied
    transformation_x = "";
    transformation_y = ""; 
    if sx!=1 or sy!=1:
        transformation_x += f"sₓ = {sx:>5.2f}"
        transformation_y += f"sᵥ = {sy:>5.2f}"
    if ax!=0 or ay!=0:
        transformation_x += ("   " if len(transformation_x)!=0 else "") + f"aₓ = {ax:>5.2f}"
        transformation_y += ("   " if len(transformation_y)!=0 else "") + f"aᵥ = {ay:>5.2f}"
    if tx!=0 or ty!=0:
        transformation_x += ("   " if len(transformation_x)!=0 else "") + f"tₓ = {tx:>5.0f}"
        transformation_y += ("   " if len(transformation_y)!=0 else "") + f"tᵥ = {ty:>5.0f}"
    
    rotationH_header = ""; 
    rotationH_x = ""; 
    rotationH_y = ""; 
    rotationH_z = ""; 
    rotationH_footer = "";     
    if r!=0:
        transformation_x += ("   " if len(transformation_x)!=0 else "") + f"θ = {r:>3.0f}⁰"  
        transformation_y += ("   " if len(transformation_x)!=0 else "") + f"        "   
        angle_str = f"{r:>3.0f}⁰"

        rotationH_header = " ┌                       ┐"
        rotationH_x = f" │cos({angle_str}) -sin({angle_str})  0│"
        rotationH_y = f" │sin({angle_str})  cos({angle_str})  0│"
        rotationH_z =  " │      0          0    1│"
        rotationH_footer = " └                       ┘"

    text_content = [
        transformation_x,
        transformation_y,
        "",
        "Homogenous",
        f"┌                     ┐"                +rotationH_header+" ┌ ┐",
        f"│{sx:>5.2f}   {ax:>5.2f}   {tx:>5.0f}│" +rotationH_x+" │x│",
        f"│{ay:>5.2f}   {sy:>5.2f}   {ty:>5.0f}│" +rotationH_y+" │y│",
        f"│    0       0       1│"                +rotationH_z+" │1│",
        f"└                     ┘"                +rotationH_footer+" └ ┘"
    ]
    
    axs[0,0].axis("off")
    axs[0,0].text(0.5, 0.6, "\n".join(text_content), 
                  fontsize=12, 
                  ha='center', va='center',
                  transform=axs[0,0].transAxes,
                  fontfamily='monospace',
                  linespacing=1.5)
    
    axs[1,0].axis("off")
    axs[1,0].text(0.5, 0.6, "\nImage Loaded:"+ img_path if img_loaded else "Default image" , 
                fontsize=12, 
                ha='center', va='center',
                transform=axs[1,0].transAxes,
                fontfamily='monospace',
                linespacing=1.5)
    plt.show()

In [ ]:
# Sliders
sx = FloatSlider(value=1, min=-5, max=5, step=0.5, description='Scale X')
sy = FloatSlider(value=1, min=-5, max=5, step=0.5, description='Scale Y')
ax = FloatSlider(value=0, min=-5, max=5, step=0.5, description='Shear X')
ay = FloatSlider(value=0, min=-5, max=5, step=0.5, description='Shear Y')
tx = FloatSlider(value=0, min=-1000, max=1000, step=100, description='Translate X')
ty = FloatSlider(value=0, min=-1000, max=1000, step=100, description='Translate Y')
r = FloatSlider(value=0, min=0, max=360, step=10, description='Rotate')
img_path = Text(value='test.png', description='Image Path')

# Arrange layout
row1 = HBox([sx, sy])
row2 = HBox([ax, ay])
row3 = HBox([tx, ty])
row4 = HBox([r, img_path])
ui = VBox([row1, row2, row3, row4])

out = interactive_output(
    manipulate_image,
    {'img_path': img_path, 'sx': sx, 'sy': sy, 'ax': ax, 'ay': ay,
     'tx': tx, 'ty': ty, 'r': r}
)

VBox([ui, out])

# **3D Transformation**

In [ ]:
def visualise_3d(
    sx=1,sy=1,sz=1,
    tx=0,ty=0,tz=0,
    axy=0,axz=0,
    ayx=0,ayz=0,
    azx=0,azy=0,
    rx=0,  # Rotation about X-axis (YZ Plane spins)
    ry=0,  # Rotation about Y-axis (XZ Plane spins)
    rz=0   # Rotation about Z-axis (XY Plane spins)
):
    # Base cube to modify
    cube = np.array([
        [0,0,0,1],
        [1,0,0,1],
        [1,1,0,1],
        [0,1,0,1],
        [0,0,1,1],
        [1,0,1,1],
        [1,1,1,1],
        [0,1,1,1]
    ]).T

    # Edge list
    edges = [
        (0,1),(1,2),(2,3),(3,0),  # bottom square
        (4,5),(5,6),(6,7),(7,4),  # top square
        (0,4),(1,5),(2,6),(3,7)   # verticals
    ]
    
    # Colours for orientation
    edge_colors = {
        (0,1): 'tomato',        # x-axis 
        (0,3): 'lawngreen',     # y-axis 
        (0,4): 'deepskyblue'   # z-axis 
        # (2,6): "mediumslateblue", #z-axis (other)
        # (5,6): "seagreen",     #y-axis (other)
        # (6,7): "sandybrown"       #x-axis (other)
    }

    # (1) Create Homogeneous Matrix 
    T_mtx = np.array([
        [ sx, axy, axz, tx],
        [ayx,  sy, ayz, ty],
        [azx, azy,  sz, tz],
        [  0,   0,   0,  1]
    ])

    R_mtx = np.eye(4)                   # Identity matrix

    if rx != 0:                         # Rotate about X-axis first
        cos_t = np.cos(np.radians(rx))
        sin_t = np.sin(np.radians(rx))
        Rx_mtx = np.array([
            [1, 0, 0, 0],
            [0, cos_t, -sin_t, 0],
            [0, sin_t,  cos_t, 0],
            [0, 0, 0, 1]
        ])
        R_mtx = Rx_mtx @ R_mtx

    if ry != 0:                         # Rotate about Y-axis second
        cos_t = np.cos(np.radians(ry))
        sin_t = np.sin(np.radians(ry))
        Ry_mtx = np.array([
            [ cos_t, 0, sin_t, 0],
            [     0, 1,     0, 0],
            [-sin_t, 0, cos_t, 0],
            [     0, 0,     0, 1]
        ])
        R_mtx = Ry_mtx @ R_mtx

    if rz != 0:                         # Rotate about Z-axis third
        cos_t = np.cos(np.radians(rz))
        sin_t = np.sin(np.radians(rz))
        Rz_mtx = np.array([
            [cos_t, -sin_t, 0, 0],
            [sin_t,  cos_t, 0, 0],
            [    0,      0, 1, 0],
            [    0,      0, 0, 1]
        ])
        R_mtx = Rz_mtx @ R_mtx
     
    # (2) Matrix Multiplication    
    cube_t = T_mtx @ R_mtx @ cube

    fig = plt.figure(figsize=(12, 7))

    # Display cube
    ax3d = fig.add_subplot(122, projection='3d')
    ax3d.set_title("Transformed Cube")
    ax3d.set_xlim(-2, 3)
    ax3d.set_ylim(-2, 3)
    ax3d.set_zlim(-2, 3)
    ax3d.set_box_aspect([1,1,1])
    ax3d.set_xlabel("X")
    ax3d.set_ylabel("Y")
    ax3d.set_zlabel("Z")

    for e in edges:
        ax3d.plot(*zip(cube[:3, e[0]], cube[:3, e[1]]), color='lightgray', lw=1, alpha=0.5) # original cube
        color = edge_colors.get(e, edge_colors.get((e[1], e[0]), 'black'))                  # transformed cube
        ax3d.plot(*zip(cube_t[:3, e[0]], cube_t[:3, e[1]]), color=color, lw=2, marker='o', markersize=4)
    
    # Display axes at origin
    axis_length = 1.5
    ax3d.quiver(0, 0, 0, axis_length, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=2, label='X-axis')
    ax3d.quiver(0, 0, 0, 0, axis_length, 0, color='green', arrow_length_ratio=0.1, linewidth=2, label='Y-axis')
    ax3d.quiver(0, 0, 0, 0, 0, axis_length, color='blue', arrow_length_ratio=0.1, linewidth=2, label='Z-axis')

    # Display Transformation Applied
    ax2d = fig.add_subplot(121)  # 1 row, 2 columns, first subplot
    ax2d.axis("off")

    transformation_x = ""
    transformation_y = ""
    transformation_z = ""
    gap = ""
    if sx!=1 or sy!=1 or sz!=1:
        transformation_x += f"sₓ = {sx:>4.1f}" if sx!=1 else "         "
        transformation_y += f"sᵥ = {sy:>4.1f}" if sy!=1 else "         "
        transformation_z += f"s₂ = {sz:>4.1f}" if sz!=1 else "         "
        gap = "   "
    if axy!=0 or ayx!=0 or azx!=0:
        transformation_x = transformation_x + gap + (f"aₓᵥ = {axy:>4.1f}" if axy!=0 else "         ")
        transformation_y = transformation_y + gap + (f"aᵥₓ = {ayx:>4.1f}" if ayx!=0 else "         ")
        transformation_z = transformation_z + gap + (f"a₂ₓ = {azx:>4.1f}" if azx!=0 else "         ")   
        gap = "   "
    if axz!=0 or ayz!=0 or azy!=0:
        transformation_x = transformation_x + gap + (f"aₓ₂ = {axz:>4.1f}" if axz!=0 else "         ")
        transformation_y = transformation_y + gap + (f"aᵥ₂ = {ayz:>4.1f}" if ayz!=0 else "         ")
        transformation_z = transformation_z + gap + (f"a₂ᵥ = {azy:>4.1f}" if azy!=0 else "         ")  
        gap = "   "
    if tx!=0 or ty!=0 or tz!=0:
        transformation_x = transformation_x + gap + (f"tₓ = {tx:>4.1f}" if tx!=0 else "         ")
        transformation_y = transformation_y + gap + (f"tᵥ = {ty:>4.1f}" if ty!=0 else "         ")
        transformation_z = transformation_z + gap + (f"t₂ = {tz:>4.1f}" if tz!=0 else "         ")   
        gap = "   "    

    has_rotation = rx!=0 or ry!=0 or rz!=0
    matrix_gap, matrix_R = "", ""
    rotation_header, rotation_footer, rotation_x, rotation_y, rotation_z, rotation_a = "","","","","",""
    rotation_header2, rotation_footer2, rotation_x2, rotation_y2, rotation_z2, rotation_a2 = "","","","","",""
    rgap = ""
    if rx!=0 or ry!=0 or rz!=0:
        if rx!=0:
            angle = f"{rx:>3.0f}⁰"
            transformation_x = transformation_x + gap + f"θₓ = {angle}"
            rotation_header = f"     ┌                        ┐"
            rotation_x      = f"     │1       0          0   0│"
            rotation_y      = f"Rₓ = │0 cos({angle}) -sin({angle}) 0│"
            rotation_z      = f"     │0 sin({angle})  cos({angle}) 0│"
            rotation_a      = f"     │0       0          0   1│"
            rotation_footer = f"     └                        ┘"
            rgap = "  "

            matrix_R = " Rₓ"
            matrix_gap = "   "
        else:
            transformation_x = transformation_x + gap + "         "
        
        if ry!=0:
            angle = f"{ry:>3.0f}⁰"
            transformation_y = transformation_y + gap + f"θᵥ = {angle}"
            rotation_header = rotation_header + rgap + f"     ┌                        ┐"
            rotation_x      = rotation_x + rgap      + f"     │ cos({angle}) 0 sin({angle}) 0│"
            rotation_y      = rotation_y + rgap      + f"Rᵥ = │       0   1       0   0│"
            rotation_z      = rotation_z + rgap      + f"     │-sin({angle}) 0 cos({angle}) 0│"
            rotation_a      = rotation_a + rgap      + f"     │       0   0       0   1│"
            rotation_footer = rotation_footer + rgap + f"     └                        ┘"
            rgap = "   "
            
            matrix_R = " Rᵥ" + matrix_R
            matrix_gap += "   "
        else:
            transformation_y = transformation_y + gap + "         "

        if rz!=0:
            angle = f"{rz:>3.0f}⁰"
            transformation_z = transformation_z + gap + f"θ₂ = {angle}"
            if rx!=0 and ry!=0:
                rotation_header2 = f"     ┌                        ┐"
                rotation_x2      = f"     │ cos({angle}) 0 sin({angle}) 0│"
                rotation_y2      = f"R₂ = │       0   1       0   0│"
                rotation_z2      = f"     │-sin({angle}) 0 cos({angle}) 0│"
                rotation_a2      = f"     │       0   0       0   1│"
                rotation_footer2 = f"     └                        ┘"

                matrix_R = " R₂" + matrix_R
                matrix_gap += "   "
            else:
                rotation_header = rotation_header + rgap + f"     ┌                        ┐"
                rotation_x      = rotation_x + rgap      + f"     │cos({angle}) -sin({angle}) 0 0│"
                rotation_y      = rotation_y + rgap      + f"R₂ = │sin({angle})  cos({angle}) 0 0│"
                rotation_z      = rotation_z + rgap      + f"     │      0          0   0 0│"
                rotation_a      = rotation_a + rgap      + f"     │      0          0   0 1│"
                rotation_footer = rotation_footer + rgap + f"     └                        ┘"
        else:
            transformation_z = transformation_z + gap + "         "

    text_content = [
        "Transformation:",
        transformation_x,
        transformation_y,
        transformation_z,
        "",
        "Homogenous",
        f"┌                       ┐"                           +matrix_gap+" ┌ ┐",
        f"│{ sx:>4.1f}  {axy:>4.1f}  {axz:>4.1f}  {tx:>5.0f}│" +matrix_gap+" │x│",
        f"│{ayx:>4.1f}  { sy:>4.1f}  {ayz:>4.1f}  {ty:>5.0f}│" +matrix_R+" │y│",
        f"│{azx:>4.1f}  {azy:>4.1f}  { sz:>4.1f}  {tz:>5.0f}│" +matrix_gap+" │z│",
        f"│   0     0     0      1│"                           +matrix_gap+" │1│",
        f"└                       ┘"                           +matrix_gap+" └ ┘",
        "Rotation" if has_rotation else "",
        rotation_header,
        rotation_x,
        rotation_y,
        rotation_z,
        rotation_a,
        rotation_footer,
        "",
        rotation_header2,
        rotation_x2,
        rotation_y2,
        rotation_z2,
        rotation_a2,
        rotation_footer2
    ] 
    ax2d.text(0.5, 0.6, "\n".join(text_content), 
                fontsize=15, 
                ha='center', va='center',
                transform=ax2d.transAxes,
                fontfamily='monospace',
                linespacing=1.5)
    
    plt.show()

In [ ]:
# Sliders
sx  = FloatSlider(value=1, min=-2, max=2, step=0.1, description='Scale X')
sy  = FloatSlider(value=1, min=-2, max=2, step=0.1, description='Scale Y')
sz  = FloatSlider(value=1, min=-2, max=2, step=0.1, description='Scale Z')
axy = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear XY Line')
axz = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear XZ Line')
ayx = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear YX Line')
ayz = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear YZ Line')
azx = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear ZX Line')
azy = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Shear ZY Line')
tx  = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Translate X')
ty  = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Translate Y')
tz  = FloatSlider(value=0, min=-2, max=2, step=0.1, description='Translate Z')
rx  = FloatSlider(value=0, min=0, max=360, step=10, description='Rotate X')
ry  = FloatSlider(value=0, min=0, max=360, step=10, description='Rotate Y')
rz  = FloatSlider(value=0, min=0, max=360, step=10, description='Rotate Z')

# Arrange layout
col1 = VBox([sx, axy, axz, tx, rx])
col2 = VBox([sy, ayx, ayz, ty, ry])
col3 = VBox([sz, azx, azy, tz, rz])
ui = HBox([col1, col2, col3])

out = interactive_output(
    visualise_3d,
    {'sx': sx, 'sy': sy, 'sz': sz,
     'axy': axy, 'axz': axz, 'ayx': ayx, 'ayz': ayz, 'azx': azx, 'azy': azy,
     'tx': tx, 'ty': ty, 'tz': tz,
     'rx': rx, 'ry': ry, 'rz': rz}
)

VBox([ui, out])